In [ ]:
from ase.md.langevin import Langevin
from ase.md import MDLogger
from ase.build import molecule
from ase import Atoms
from xtb.ase.calculator import XTB
from ase.calculators.emt import EMT
from ase import units
from rich import print
from ase.calculators.acn import ACN, m_me, r_cn, r_mec
from ase.constraints import FixLinearTriatomic
from ase.io import Trajectory
import numpy as np

In [ ]:
from xtb.utils import get_method
get_method('GFN2-xTB') is not None

In [ ]:
# Bond lengths
from ase.data.pubchem import pubchem_atoms_search, pubchem_atoms_conformer_search
# Create butene molecule
# atoms = Atoms('CCCCHHHHHHHH', positions=positions)
# atoms.rotate(30, 'x')
atoms = pubchem_atoms_search(smiles='CC=CC')
# Now use this butene object in your MD simulation

In [ ]:
print(atoms)

In [ ]:
d = 0.776 / 1e24
L = ((atoms.get_masses().sum() / units.mol) / d)**(1 / 3.) * 10
# Set up box of 27 acetonitrile molecules
atoms.set_cell((L, L, L))
atoms.center()
# atoms = atoms.repeat((3, 3, 3))
print(atoms)

In [ ]:
atoms.calc = XTB(
    method="GFN2-xTB",
    directory='xtb_files',
    atoms=atoms,
    charge=0,
    multiplicity=1,
    parallel=True,
    periodic=True,
)
print(atoms.get_potential_energy())
print(atoms.get_forces())

In [ ]:
atoms.calc

In [ ]:
dyn = Langevin(
    atoms,
    timestep=0.5 * units.fs,
    temperature_K=298,
    friction=0.02,
    trajectory='md1.traj',
    logfile='md1.log',
    append_trajectory=True)


# Set up trajectory
traj = Trajectory('md.traj', 'w', atoms)
dyn.attach(traj.write, interval=1)
# dyn.attach(MDLogger(
#     dyn,
#     atoms,
#     'md.log',
#     header=False,
#     stress=False,
#     peratom=True,
#     mode="a"
#     ), interval=1000)

In [ ]:
from logmd import LogMD
logmd = LogMD()
dyn.attach(lambda: logmd(atoms), interval=1)

In [ ]:
dyn.run(10000)

In [ ]:
print(atoms.get_potential_energy())
print(atoms.get_forces())

In [ ]:
print(atoms.get_potential_energy())
print(atoms.get_forces())

In [ ]:
import plotly.express as px
import pandas as pd

# Load the data (replace 'your_file.txt' with the actual filename)
df = pd.read_csv("md.log", delim_whitespace=True)

# Plot Time vs. Etot
fig = px.line(df, x="Time[ps]", y="Etot[eV]", title="Total Energy vs. Time")

# Show the interactive plot
fig.show()

In [ ]:
import plotly.express as px
import pandas as pd

# Load the data (replace 'your_file.txt' with the actual filename)
df = pd.read_csv("md.log", delim_whitespace=True)

# Plot Time vs. Etot
fig = px.line(df, x="Time[ps]", y="T[K]", title="T[K] vs. Time")

# Show the interactive plot
fig.show()

In [ ]:
!ls

In [ ]:
from ase.io import read
from ase.visualize import view

# Read the entire trajectory
traj = read("md.traj", index=":")  # ":" means all frames
view(traj)  # Opens a simple GUI viewer

In [ ]:
# Pull out specific snapshots (e.g., the last frame):
last_frame = read("mymd.traj", index=-1)  # Last frame
print(last_frame.get_positions())  # Atomic coordinates

In [ ]:
from ase.io import write
write("mymd.xyz", traj)  # Creates a multi-frame XYZ file

In [ ]:
for frame in traj:
    energy = frame.get_potential_energy()
    print(f"Energy: {energy} eV")